In [43]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import dask

from functools import partial

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
topdir="/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/monthly/RHO/"

In [123]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '50GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='03:00:00',
    interface='ib0')

# scale up
cluster.scale(30)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46421 instead
  http_address["port"], self.http_server.port


In [124]:
client

Client Scheduler: tcp://10.12.206.60:43217 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/46421/status,Cluster Workers: 28 Cores: 28 Memory: 601.30 GB


In [125]:
dat = xr.open_mfdataset("/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/monthly/RHO/"+
                      "b.e11.B1850C5CN.f09_g16.005.pop.h.RHO.*.nc", 
                       chunks={"time":300, "z_t":60, "nlat":30, "nlon":30},
                       concat_dim='time', combine='by_coords', coords='minimal')

In [126]:
timebnds = dat.time_bound
tarea = dat.TAREA
dz = dat.dz
z_t = dat.z_t
tarea = dat.TAREA
dat = dat.RHO
diff = np.array(timebnds.isel(d2=1)) - np.array(timebnds.isel(d2=0))
diff = diff/2.
newtime = np.array(timebnds.isel(d2=0)) + diff
dat['time'] = newtime
dz['time'] = newtime
tarea['time'] = newtime
dat = dat[dat.time.dt.month == 3]
dz = dz[dz.time.dt.month == 3]
dat = dat.isel(z_t=slice(0,20))
z_t = z_t.isel(z_t=slice(0,20))
dz = dz.isel(z_t=slice(0,20))

In [127]:
dat = dat.chunk(chunks={"time":300, "z_t":60, "nlat":30, "nlon":30})
dz = dz.chunk(chunks={"time":300, "z_t":60})
tarea = tarea.chunk(chunks={"time":300, "nlat":30, "nlon":30})

In [128]:
rho = (dat*dz).sum('z_t')/dz.sum('z_t')

In [129]:
wgts = tarea
wgts = xr.where( (wgts.TLONG > 300) & (wgts.TLONG < 325), wgts, 0)
wgts = xr.where( (wgts.TLAT > 50) & (wgts.TLAT < 65), wgts, 0)

In [130]:
rho_w = rho.weighted(wgts.fillna(0))
rhom = rho_w.mean(("nlon","nlat"))

In [131]:
rhom = rhom.load()

In [133]:
rhom = rhom.rename('RHO')

In [134]:
rhom.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/cesm1_picontrol/cesm1_picontrol_rho_top203_60to35W_50to65N.nc")

In [135]:
cluster.close()